In [49]:
#Reset Listes (Pour redémarerentièrement le programme on vide les liste de noms et d'url)

liste_name = []
liste_url_pages = []
print(liste_name)
print(liste_url_pages)

[]
[]


In [50]:
from bs4 import BeautifulSoup
import requests
import os
import time
 
if 'liste_name' in globals():
    List_nom = liste_name
else:
    List_nom = []

if 'liste_url_pages' in globals():
    List_url = liste_url_pages
else:
    List_url = []

    
def getImgLink(url):
    
    html_page = requests.get(url)
    soup = BeautifulSoup(html_page.text, 'html.parser')
    url_images = [i.find('img').get('src') for i in soup.find_all('a', class_='image')]
    url_page_images = [i.get('href') for i in soup.find_all('a', class_='image')]
    url_images = ['/'.join(i.split('/')[:-1]).replace('/thumb', '') for i in url_images] #On récupère les fichiers d'origine et non les thumbnails
    
    return url_images,url_page_images

def saveImg(url_images,name,url_pages): #Fonction qui télécharge N images jpg et png
    cpt = 0
    num = 1
    volume = int(input("Combien d'image voulez vous ? "))
    while cpt < volume:
        time.sleep(1)
        image = url_images[cpt]
        ext = os.path.splitext(image)[-1].lower()
        if ext == '.jpg' or ext == '.jpeg':
            chemin ='/home/thomas/Projet/data/images_projet/' #Bien modifier le chemin de téléchargement
            chemin = chemin + name + str(num) + str(ext)
            f = open(chemin,'wb')
            f.write(requests.get(image).content)
            f.close()
            List_nom.append(chemin)
            List_url.append(url_pages[cpt])
            num+=1
        else:
            volume+=1
        cpt+=1
        
    return List_url,List_nom  
    
    
Search_Word = input("Entrez un mot cle a rechercher :")

liste_images,liste_url_pages = getImgLink('https://commons.wikimedia.org/w/index.php?title=Special:Search&limit=500&search='+Search_Word)

liste_url_pages, liste_name = saveImg(liste_images,Search_Word,liste_url_pages)

print("Done")

pizza
65
Done


In [60]:
from bs4 import BeautifulSoup
import requests
import os
import os.path
import PIL.Image
import random
import json

existingTagList = []
DataImage = {}
numImage = 0
for pathImg in liste_name:
    print(os.path.splitext(pathImg)[0])
    img = PIL.Image.open(pathImg)
    exif_data = img._getexif()
    
    #Nom fichier
    Nom = os.path.splitext(pathImg)[0].lstrip("/home/thomas/Projet/data/images_projet/")
    
    #Format
    Format = os.path.splitext(pathImg)[1]

    #Taille image
    if img.size[0]*img.size[1] > 750000:
        Taille = "Grande"
    elif img.size[0]*img.size[1] < 75000:
        if img.size[0]==img.size[1]:
            Taille = "Vignette"
        else:
            Taille = "Petite"
    else:
        Taille = "Moyenne"
    
    if exif_data is not None:
        
        #Date création
        if 306 in exif_data:
            creationDate = exif_data[306] 
        else:
            creationDate = None
        
        #Marque appareil
        if 271 in exif_data:
            marqueAppareil = exif_data[271]
        else:
            marqueAppareil = None
        
        #Modèle appareil
        if 272 in exif_data:
            modeleAppareil = exif_data[272]
        else:
            modeleAppareil = None
        
        #Sens de l'image
        if 274 in exif_data:
            numSensImage = exif_data[274]
            #1, 2, 3 ou 4 --> Horizontal / 5, 6, 7 ou 8 --> Vertical
            if numSensImage <= 4:
                sensImage = "Horizontal"
            else:
                sensImage = "Vertical"
        else:
            sensImage = None
    else :
        creationDate = None
        marqueAppareil = None
        modeleAppareil = None
        sensImage = None

    #Tag des images
    choix = "N"
    imageTagList = []
    html_page = requests.get("https://commons.wikimedia.org/" + liste_url_pages[numImage])
    soup = BeautifulSoup(html_page.text, 'html.parser')
    
    if soup.findAll('div', {'id' : 'mw-normal-catlinks'}):
        for category in soup.findAll('div', {'id' : 'mw-normal-catlinks'})[0].findChildren("a")[1:]:
            Tag = category.attrs['title'].split(':')[1]
            imageTagList.append(Tag)
            if Tag not in existingTagList:
                existingTagList.append(Tag)
        
    
    #Pour ajouter des tags manuellement, il suffit de décommenter la ligne suivante
    #choix = input("Voulez vous ajouter un tag à cette image ? (Y/N)")
    while choix == "Y" or choix == "y":
        Tag = input("Entrez un tag que vous voulez associer à cette image : ")
        Tag = Tag.lower()
        if Tag not in imageTagList:
            imageTagList.append(Tag)
            if Tag not in existingTagList:
                existingTagList.append(Tag)    
        choix = input("Voulez vous ajouter un autre tag à cette image ? (Y/N)")
            
    #Stockage dans un fichier json
    Dico_Image = {
        "nom":Nom,
        "format":Format,
        "taille":Taille,
        "date_creation":creationDate,
        "marque":marqueAppareil,
        "modele":modeleAppareil,
        "orientation":sensImage,
        "tags":imageTagList
        }
    
    DataImage["Image " + str(numImage+1)] = Dico_Image
    
    numImage+=1

with open("DataImage.json","w") as f:
    json.dump(DataImage, f)

#print(existingTagList)


/home/thomas/Projet/data/images_projet/pizza1
/home/thomas/Projet/data/images_projet/pizza2
/home/thomas/Projet/data/images_projet/pizza3
/home/thomas/Projet/data/images_projet/pizza4
/home/thomas/Projet/data/images_projet/pizza5
/home/thomas/Projet/data/images_projet/pizza6
/home/thomas/Projet/data/images_projet/pizza7
/home/thomas/Projet/data/images_projet/pizza8
/home/thomas/Projet/data/images_projet/pizza9
/home/thomas/Projet/data/images_projet/pizza10
/home/thomas/Projet/data/images_projet/pizza11
/home/thomas/Projet/data/images_projet/pizza12
/home/thomas/Projet/data/images_projet/pizza13
/home/thomas/Projet/data/images_projet/pizza14
/home/thomas/Projet/data/images_projet/pizza15
/home/thomas/Projet/data/images_projet/pizza16
/home/thomas/Projet/data/images_projet/pizza17
/home/thomas/Projet/data/images_projet/pizza18
/home/thomas/Projet/data/images_projet/pizza19
/home/thomas/Projet/data/images_projet/pizza20
/home/thomas/Projet/data/images_projet/pizza21
/home/thomas/Projet/da

In [61]:
import json


dicoAnnee = {}
dicoModele = {}
dicoTaille = {}
dicoOrientation = {}


with open('DataImage.json') as json_data:
    data_dict = json.load(json_data)

#On parcours le fichier json pour récupérer les données intéresssantes et on les stocke dans de nouveaux dictionnaires
#Si ils n'existent pas on les ajoute, sinon on incrémente leur compteur (valeur)
for Image in data_dict.items():
    if Image[1]["date_creation"] != None:
        if Image[1]["date_creation"][:4] not in dicoAnnee:
            dicoAnnee[Image[1]["date_creation"][:4]] = 1
        else:
            dicoAnnee[Image[1]["date_creation"][:4]] += 1
                    
    if Image[1]["taille"] != None:
        if Image[1]["taille"] not in dicoTaille:
            dicoTaille[Image[1]["taille"]] = 1
        else:
            dicoTaille[Image[1]["taille"]] += 1

    if Image[1]["modele"] != None:
        if Image[1]["modele"] not in dicoModele:
            dicoModele[Image[1]["modele"]] = 1
        else:
            dicoModele[Image[1]["modele"]] += 1

    if Image[1]["orientation"] != None:
        if Image[1]["orientation"] not in dicoOrientation:
            dicoOrientation[Image[1]["orientation"]] = 1
        else:
            dicoOrientation[Image[1]["orientation"]] += 1

print(dicoAnnee)
print(dicoModele)
print(dicoTaille)
print(dicoOrientation)

{'2018': 9, '2005': 8, '2014': 3, '2008': 1, '2016': 3, '2020': 3, '2019': 4, '2010': 2, '2011': 2, '2017': 4, '2015': 1, '2012': 2, '2006': 2, '2009': 2, '2002': 1}
{'SM-S907VL': 1, 'Canon PowerShot A40': 3, 'Canon EOS 100D': 1, 'COOLPIX P900': 1, 'P8_Mini\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00': 2, 'SM-G965U': 1, 'DSC-RX100M3': 1, 'Canon PowerShot A75': 4, 'Canon PowerShot G9': 1, 'Pixel 2 XL': 4, 'LG-P970': 2, 'realme 3 Pro': 1, 'X-E1': 1, 'Canon EOS 50D': 1, 'SM-G960U': 1, 'ILCE-7M2': 2, 'DSC-S60': 1, 'SM-A207F': 1, 'Canon EOS 450D': 1, 'Canon EOS 6D': 1, 'FinePix F50fd  ': 1, 'DMC-S3': 1, 'FinePix2650   ': 1, 'SM-N950U': 1, 'Canon PowerShot ELPH 350 HS': 1, 'Canon PowerShot SD1000': 1, 'Canon PowerShot SX280 HS': 1, 'NIKON D60': 1, 'FinePix S5500  ': 2, 'GT-N7100': 1, 'CLT-L29': 1, 'Canon DIGITAL IXUS 500': 1}
{'Grande': 59, 'Moyenne': 5, 'Petite': 1}
{'Horizontal': 36, 'Vertical': 2}


In [62]:
import random
import json

#On simule un snsemble de like d'un utilisateur de manière aléatoire (1/3 des images likés)

DataLike = {}
LikeMax = len(data_dict)/3
nbLike = 0
numImage = 1

while nbLike < LikeMax:
    Like = bool(random.getrandbits(1))
    if Like == True:
        DataLike["Image " + str(numImage)] = True
        nbLike+=1
    else:
        DataLike["Image " + str(numImage)] = False
    numImage+=1
    
for i in range(len(data_dict)-numImage+1):
    DataLike["Image " + str(numImage)] = False
    numImage+=1

with open("DataLike.json","w") as f:
    json.dump(DataLike, f)

In [63]:
import json


#On parcours le fichier json pour récupérer les tags intéressants pour l'utilisateur

dicoTag = {}


with open('DataLike.json') as json_like_data:
    like_dict = json.load(json_like_data)
    
with open('DataImage.json') as json_data:
    data_dict = json.load(json_data)

for Image in like_dict.items():
    if Image[1] == True:
        cpt = 0
        for tag in data_dict[Image[0]]["tags"]:
            if data_dict[Image[0]]["tags"][cpt] not in dicoTag:
                dicoTag[data_dict[Image[0]]["tags"][cpt]] = 1
            else:
                dicoTag[data_dict[Image[0]]["tags"][cpt]] += 1
            cpt += 1

#On détermine le meilleur tag
maxi = None
for tag in dicoTag:
    if maxi is None or dicoTag[tag] > maxi:
        maxi = dicoTag[tag]
        bestTag = tag
print(bestTag)

#Image que l'utilisateur pourrait aimer
for Image in data_dict.items():
    for tag in Image[1]["tags"]:
        if tag == bestTag:
            print("Vous pourriez aimer cette image : ")
            print(Image[0] + " : " + Image[1]["nom"] + Image[1]["format"])

Pizzas in the United States
Vous pourriez aimer cette image : 
Image 1 : zza1.jpg
Vous pourriez aimer cette image : 
Image 7 : zza7.jpg
Vous pourriez aimer cette image : 
Image 8 : zza8.jpg
Vous pourriez aimer cette image : 
Image 10 : zza10.jpg
Vous pourriez aimer cette image : 
Image 29 : zza29.jpg
Vous pourriez aimer cette image : 
Image 35 : zza35.jpg
Vous pourriez aimer cette image : 
Image 43 : zza43.jpg
Vous pourriez aimer cette image : 
Image 49 : zza49.jpg
